In [11]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [12]:


# Amazon product URL
url = "https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ"

# Headers to mimic a real browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# Send GET request
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Extract product title
    title = soup.find("span", id="productTitle")
    product_name = title.get_text(strip=True) if title else "Title not found"
    
    # Extract product rating
    rating = soup.find("span", class_="a-icon-alt")
    product_rating = rating.get_text(strip=True) if rating else "Rating not found"
    
    # Create timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Write data to CSV file
    csv_filename = "AmazonWebScrapping.csv"
    with open(csv_filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, product_name, product_rating])
    
    # Read and display the CSV contents
    with open(csv_filename, mode='r', encoding='utf-8') as file:
        print(file.read())
    
    print(f"Data saved to {csv_filename}")
else:
    print("Failed to retrieve the page. Check your headers or try again later.")


2025-04-02 10:41:34,Title not found,Rating not found
2025-04-02 10:41:52,Title not found,Rating not found
2025-04-02 10:44:26,Got Data Funny Business Data Analyst T-Shirt,4.4 out of 5 stars
2025-04-02 10:45:31,Got Data Funny Business Data Analyst T-Shirt,4.4 out of 5 stars
2025-04-02 10:45:49,Title not found,Rating not found

Data saved to AmazonWebScrapping.csv
